# Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

import sys
sys.path.insert(0, '../../')
from plot_graphs import plot_loss, plot_acc

# Setting Image and Batch Size

In [2]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

# Loading the train, val, and test data

In [3]:
train = utils.image_dataset_from_directory("../../datasets/data/split/VTID2/train", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("../../datasets/data/split/VTID2/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("../../datasets/data/split/VTID2/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3048 files belonging to 5 classes.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Found 652 files belonging to 5 classes.
Found 656 files belonging to 5 classes.


2022-08-03 13:58:27.814527: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-03 13:58:27.814653: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Defining the model

In [4]:
inputs = keras.Input(shape=(100, 100, 3))
x = inputs
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(16, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(5, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop(0.005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                         

# Adding Callbacks

In [5]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=8
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="../../models/vtid2/m3_small.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

# Running the model

In [6]:
history = model.fit(
  train.cache(),
  epochs=50,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/50


2022-08-03 13:58:28.668422: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-03 13:58:28.668511: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


96/96 [==============================] - ETA: 0s - loss: 1.5552 - accuracy: 0.3107

2022-08-03 13:58:33.105194: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


96/96 [==============================] - 5s 31ms/step - loss: 1.5552 - accuracy: 0.3107 - val_loss: 1.6082 - val_accuracy: 0.2853
Epoch 2/50
96/96 [==============================] - 2s 23ms/step - loss: 1.4677 - accuracy: 0.3930 - val_loss: 1.6184 - val_accuracy: 0.2929
Epoch 3/50
96/96 [==============================] - 2s 22ms/step - loss: 1.3844 - accuracy: 0.4488 - val_loss: 1.5082 - val_accuracy: 0.3344
Epoch 4/50
96/96 [==============================] - 2s 22ms/step - loss: 1.3069 - accuracy: 0.4833 - val_loss: 1.4375 - val_accuracy: 0.3896
Epoch 5/50
96/96 [==============================] - 2s 22ms/step - loss: 1.2198 - accuracy: 0.5259 - val_loss: 1.5239 - val_accuracy: 0.3926
Epoch 6/50
96/96 [==============================] - 2s 22ms/step - loss: 1.1398 - accuracy: 0.5653 - val_loss: 1.6049 - val_accuracy: 0.4049
Epoch 7/50
96/96 [==============================] - 2s 23ms/step - loss: 1.0608 - accuracy: 0.6017 - val_loss: 1.4976 - val_accuracy: 0.4479
Epoch 8/50
96/96 [======

# Evaluating the model

In [7]:
model.evaluate(test)

load_model = keras.models.load_model("../../models/vtid2/m3_small.h5")
load_model.evaluate(test)

 1/21 [>.............................] - ETA: 4s - loss: 0.5282 - accuracy: 0.9062

2022-08-03 14:00:26.722182: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 1s 17ms/step - loss: 0.2589 - accuracy: 0.9329


[0.25887244939804077, 0.9329268932342529]

In [ ]:
# Plotting loss and accuracy graphs
## Plotting loss
plot_loss(history)
## Plotting accuracy
plot_acc(history)